這是一個資料查詢系統：輸入起始與結束時間，系統會統計這段時間各題上傳結果。
首先，系統將結果顯示為6×5的報表，4首欄依序填入Problem Status 和五種答題狀況：
Accepted, Compile Error, Runtime Error, Time Limit Exceed 與 Wrong Answer，
第一列最左邊是 Problem Status，其餘依序為Problem 1 到 Problem 4。
報表其他部分則填入對應的上傳結果的次數，例如第三欄、第四列的表格所填數字為：查詢的起始時間到結束時間內，Problem2對應到的RuntimeError 次數。
其次，由於資料視覺化正當紅，系統必須另外印出一張「並列長條圖」。5圖形標題顯示查詢的時間區段。

In [35]:
import pandas as pd
from datetime import datetime
from prettytable import PrettyTable
import plotly.express as px

In [36]:
def read_data(file_path):
    """
    讀取 CSV 檔案並返回 DataFrame。

    參數：
    file_path (str): CSV 檔案路徑

    返回：
    DataFrame: 讀取的數據框
    """
    return pd.read_csv(file_path)

In [37]:
def generate_report(start_time, end_time, df):
    """
    根據指定的時間範圍統計每個問題的提交狀態次數。

    參數：
    start_time (datetime): 起始時間
    end_time (datetime): 結束時間
    df (DataFrame): 包含提交記錄的數據框

    返回：
    DataFrame: 包含每個問題不同狀態的提交次數
    """
    # 定義問題狀態及答題狀況
    problem_status = ["Accepted", "Compile Error", "Runtime Error", "Time Limit Exceed", "Wrong Answer"]
    problems = ["Problem 1", "Problem 2", "Problem 3", "Problem 4"]

    # 初始化報表
    report = {status: {problem: 0 for problem in problems} for status in problem_status}

    # 根據時間範圍統計上傳結果次數
    for _, row in df.iterrows():
        try:
            timestamp = row['SubmissionTime']
            problem = int(row['Problem'])
            status = row['Status']
            
            # 只保留時分秒部分
            timestamp = datetime.strptime(timestamp[-8:], "%H:%M:%S")
            
            if start_time.time() <= timestamp.time() <= end_time.time():
                report[status][f"Problem {problem}"] += 1
        except (KeyError, ValueError):
            pass

    # 將報表轉換為 DataFrame
    report_df = pd.DataFrame(report)

    return report_df

In [38]:
def display_table(report_df):
    """
    使用 PrettyTable 繪製表格。

    參數：
    report_df (DataFrame): 報表數據框
    """
    table = PrettyTable()
    table.field_names = ["問題"] + list(report_df.columns)
    for index, row in report_df.iterrows():
        table.add_row([index] + list(row))
    print(table)

def plot_problem_to_states(report_df):
    """
    使用 Plotly Express 繪製柱狀圖。

    參數：
    report_df (DataFrame): 報表數據框
    """
    # 將報表 DataFrame 轉換為長格式
    report_long_df = report_df.reset_index().melt(id_vars='index', var_name='Problem Status', value_name='Count')
    # 繪製條形圖
    fig = px.bar(report_long_df, x='Problem Status', y='Count', color='index', barmode='group',
                 labels={'index': 'Problem'}, title='States Report for Each Problems')

    fig.show()
    
def plot_states_to_problem(report_df):
    # 將報表 DataFrame 轉換為長格式
    report_long_df = report_df.reset_index().melt(id_vars='index', var_name='Problem Status', value_name='Count')

    # 繪製條形圖
    fig = px.bar(report_long_df, x='index', y='Count', color='Problem Status', barmode='group',
                 labels={'index': 'Problems'}, title='Problems Report for Each States')
    fig.show()

In [39]:
def main():
    # 讀取數據
    file_path = 'midterm2.csv'
    df = read_data(file_path)

    # 輸入起始與結束時間
    start_time_str = input("請輸入起始時間 (HH:MM:SS): ") or '9:20:00'
    end_time_str = input("請輸入結束時間 (HH:MM:SS): ") or '12:50:00'

    # 將輸入的時間字串轉換為 datetime 對象
    start_time = datetime.strptime(start_time_str, "%H:%M:%S")
    end_time = datetime.strptime(end_time_str, "%H:%M:%S")

    # 生成報表數據框，以及印出表格
    report_df = generate_report(start_time, end_time, df)
    display_table(report_df)

    # 繪製柱狀圖
    plot_states_to_problem(report_df)
    plot_problem_to_states(report_df)


if __name__ == "__main__":
    main()

請輸入起始時間 (HH:MM:SS): 
請輸入結束時間 (HH:MM:SS): 
+-----------+----------+---------------+---------------+-------------------+--------------+
|    問題   | Accepted | Compile Error | Runtime Error | Time Limit Exceed | Wrong Answer |
+-----------+----------+---------------+---------------+-------------------+--------------+
| Problem 1 |    88    |       6       |       6       |         0         |      21      |
| Problem 2 |    24    |       5       |       22      |         1         |      68      |
| Problem 3 |    49    |       2       |       35      |         0         |      88      |
| Problem 4 |    62    |       4       |       65      |         0         |      81      |
+-----------+----------+---------------+---------------+-------------------+--------------+
